## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [ ]:
!nvidia-smi

In [ ]:
!killall python

Installing cuDNN & pytorch - Setting environmental variables

In [ ]:
import platform
print(platform.system())
if platform.system()== "Windows":
    %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
else:
    print('running on linux')
    %pip install torch torchvision torchaudio
    %pip install torch torchvision torchaudio -U
%pip install comet_ml --quiet
# pip install comet_ml
#%env CUDA_PATH = "E:\software\cuDNN\cudnn-windows-x86_64-8.9.5.29_cuda12-archive"
import torch
import torchvision
print(torch.cuda.__path__)
print(torch.version.cuda)

#torch.from_numpy(boxes.detach().cpu().numpy())


torch.cuda.is_available()


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## Install YOLOv8

⚠️ YOLOv8 is still under heavy development. Breaking changes are being introduced almost weekly. We strive to make our YOLOv8 notebooks work with the latest version of the library. Last tests took place on **27.01.2023** with version **YOLOv8.0.20**.

If you notice that our notebook behaves incorrectly - especially if you experience errors that prevent you from going through the tutorial - don't hesitate! Let us know and open an [issue](https://github.com/roboflow/notebooks/issues) on the Roboflow Notebooks repository.

YOLOv8 can be installed in two ways - from the source and via pip. This is because it is the first iteration of YOLO to have an official package.

In [ ]:

# Pip install method (recommended)

#%pip install ultralytics==8.0.20
#%pip  install ultralytics
%pip install ultralytics

from IPython import display
#display.clear_output()

import ultralytics
ultralytics.checks()

import torch
GPUS_AVALIBLE = torch.cuda.device_count()
print(GPUS_AVALIBLE)


In [ ]:
# Git clone method (for development)

# %cd {HOME}
# !git clone github.com/ultralytics/ultralytics
# %cd {HOME}/ultralytics
# !pip install -e .

# from IPython import display
# display.clear_output()

# import ultralytics
# ultralytics.checks()

If you want to train, validate or run inference on models and don't need to make any modifications to the code, using YOLO command line interface is the easiest way to get started. Read more about CLI in [Ultralytics YOLO Docs](https://docs.ultralytics.com/usage/cli/).

```
yolo task=detect    mode=train    model=yolov8n.yaml      args...
          classify       predict        yolov8n-cls.yaml  args...
          segment        val            yolov8n-seg.yaml  args...
                         export         yolov8n.pt        format=onnx  args...
```

In [ ]:
#%mkdir {HOME}
%cd {HOME}


import dotenv
dotenv.load_dotenv(".env")

ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")

%pip install roboflow --quiet
#%pip install ultralytics==8.0.134

from roboflow import Roboflow
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("trashclassification-tayqe").project("trash-xvysc")
dataset = project.version(2).download("yolov8",overwrite=False, location=f"{HOME}//datasets")
dataset_version = dataset.version

## Custom Training

In [ ]:
#%run scripts/train.py

#CLASSIFY requirements
"""
imgsz=224
task=classify
dataset must be heiracel or clip
dataset objects must be isolated for classification
"""

from scripts.train import YOLO_Trainer

model = "yolov8x-cls.pt"
clean_model =  model.removesuffix(".pt")

task = "classify"
imgsz = 224

cstm_class = YOLO_Trainer(model=model, task=task, classify_project_id="trash-vs-recycling-pi-cam", clearml_project_name="rpi-trash-classification")

cstm_class.train("trash-classification",task=task,bs=64,name=clean_model, exist_ok=True, export='torchscript',  imgsz = 224, cache=True, epochs=50)
#cstm_class.model.benchmark(data=cstm_class.dataset.location)

In [ ]:
from ultralytics import YOLO
import torch

#pytorchmodel = torch.jit.script("exports/best.pt")

model = YOLO("exports/pytorchtotrt.pt")
model.export(format="onnx")

In [ ]:
from ultralytics import YOLO
from torchinfo import summary
from torchvision import models



#model = YOLO("trash-classification/yolov8m/weights/best.pt")
#print(model.info(detailed=True))

#model = YOLO("trash-classification/yolov8l-cls/weights/best.pt")
model = YOLO("yolov8x-cls")
model.export(format="torchscript")
#model.benchmark(data="datasets/trash-2/clip")
# pt_model = YOLO("trash-classification/yolov8m/weights/best.pt")
# print(pt_model.info(detailed=True))
# print(pt_model)
#print(vgg)
#print(model)

In [ ]:
print(HOME)
%ls {HOME}\\runs\\detect\\train31\\
#%ls runs\detect\train31\

curr_train_path = HOME+"/runs/detect/train39"
curr_train = os.path.normpath(curr_train_path)
print(curr_train)


In [ ]:
%cd {HOME}
from IPython.display import Image
#Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)
Image(filename=f"{curr_train}/confusion_matrix.png",width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{curr_train}/results.png', width=600)

In [ ]:
%cd {HOME}
valbatch0 = Image(filename=f'{curr_train}/val_batch0_pred.jpg', width=600)
valbatch1 = Image(filename=f'{curr_train}/val_batch1_pred.jpg', width=600)
valbatch2 = Image(filename=f'{curr_train}/val_batch2_pred.jpg', width=600)
display.display(valbatch0,valbatch1,valbatch2)